In [4]:
import cv2
from ultralytics import YOLO
import os
from paddleocr import PaddleOCR, draw_ocr

class LicensePlateProcessor:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.ocr = PaddleOCR(use_angle_cls=True, lang='en')
    
    def detect_and_recognize(self, image_path, output_folder):
        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return None, "No text detected"

        # Run inference to detect license plate
        results = self.model(image)
        boxes = results[0].boxes

        if boxes is None or len(boxes) == 0:
            print(f"No license plate detected in {image_path}.")
            return None, "No text detected"

        highest_conf_box = None
        highest_conf_score = 0

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0] 
            confidence = box.conf[0]  
            cls = box.cls[0] 
            
            if int(cls) == 0: 
                if confidence > highest_conf_score:
                    highest_conf_box = {'x1': int(x1), 'y1': int(y1), 'x2': int(x2), 'y2': int(y2), 'confidence': confidence}
                    highest_conf_score = confidence
        
        # If a license plate was detected, crop it from the image
        if highest_conf_box is not None:
            x1, y1, x2, y2 = highest_conf_box['x1'], highest_conf_box['y1'], highest_conf_box['x2'], highest_conf_box['y2']
            cropped_plate = image[y1:y2, x1:x2]
            
            # Save cropped plate to output folder
            cropped_output_path = os.path.join(output_folder, f"cropped_{os.path.basename(image_path)}")
            os.makedirs(output_folder, exist_ok=True)
            cv2.imwrite(cropped_output_path, cropped_plate)

            # Perform OCR on the cropped license plate
            ocr_results = self.ocr.ocr(cropped_output_path, cls=True)
            
            # Default detected text
            detected_text = "No text detected"

            # Check if OCR results are valid
            if ocr_results and isinstance(ocr_results, list) and len(ocr_results) > 0 and ocr_results[0]:
                detected_text = " ".join([line[1][0] for line in ocr_results[0]])

            # Write detected text on the image
            cv2.putText(image, detected_text.strip(), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            # Draw the bounding box on the image
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Save the annotated image
            annotated_image_path = os.path.join(output_folder, f"annotated_{os.path.basename(image_path)}")
            cv2.imwrite(annotated_image_path, image)
            
            print(f"Annotated image saved at {annotated_image_path}")
            print(f"Detected text: {detected_text.strip()}")
            return highest_conf_box, detected_text.strip()
        
        print(f"No valid license plate detected in {image_path}.")
        return None, "No text detected"


# Example usage
model_path = '/media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/best.pt'  # Specify the path to your YOLOv8 model
processor = LicensePlateProcessor(model_path)

# Folder containing license plate images
image_folder = '/media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/test_images'  # Replace with the path to your folder
output_folder = '/media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/OCR test image'    # Replace with the desired output folder

for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    if os.path.isfile(image_path):
        bbox, detected_text = processor.detect_and_recognize(image_path, output_folder)
        print(f"Processed: {image_file}")
        print("Bounding box:", bbox)
        print("Detected text:", detected_text)


[2024/11/18 20:06:10] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/asmany/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/asmany/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_le


0: 640x640 (no detections), 75.2ms
Speed: 4.0ms preprocess, 75.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
No license plate detected in /media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/test_images/5daafbcf76fa6602_jpg.rf.14442dd74aa0e629fe892e73fb65e71f_fog.jpg.
Processed: 5daafbcf76fa6602_jpg.rf.14442dd74aa0e629fe892e73fb65e71f_fog.jpg
Bounding box: None
Detected text: No text detected

0: 640x640 1 license_plate, 71.8ms
Speed: 2.8ms preprocess, 71.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
[2024/11/18 20:06:11] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.02639007568359375
[2024/11/18 20:06:11] ppocr DEBUG: cls num  : 1, elapsed : 0.015416860580444336
[2024/11/18 20:06:11] ppocr DEBUG: rec_res num  : 1, elapsed : 0.14766311645507812
Annotated image saved at /media/asmany/e054e025-4de0-482e-bf54-4fb67a1e5566/Hof University/AI Project/OCR test image/annotated_OUP1442_rain.jpg
Detected text: OUP1442
Processed: